# Smoothing

Here are three different operators that smooth data.  


## Causal integration

The first operator is a triangular matrix filled with 1s. 

Note
- how we use recursion to apply the operator
- the form of the adjoint
   

In [ ]:
%load_ext autoreload
%autoreload 2
import sys

! pip install  "sep_plot @ git+https://github.com/SEP-software/sep-plot.git@3fac86108f59c822193cbd6f28687fecce5e298b" 
import generic_solver


  Cloning https://github.com/SEP-software/sep-plot.git (to revision 3fac86108f59c822193cbd6f28687fecce5e298b) to /private/var/folders/69/z1qrsb455x90d4pvp67r4ccr00lgnl/T/pip-install-4gsich2e/sep-plot_2cd00b36a87a49e6a70f84d7b2f9d3df
  Running command git clone -q https://github.com/SEP-software/sep-plot.git /private/var/folders/69/z1qrsb455x90d4pvp67r4ccr00lgnl/T/pip-install-4gsich2e/sep-plot_2cd00b36a87a49e6a70f84d7b2f9d3df


In [ ]:
from generic_solver._pyOperator import Operator
from sep_python import FloatVector
import numba

class Causal(Operator):

    def __init__(self, mod, dat):
        """
        Initialize a transient convolition

            mod, dat - sepVector
        """
        super().__init__(mod, dat)
        if not isinstance(mod, FloatVector) or not isinstance(dat,FloatVector):
            raise Exception("Expecting model and data to be sepVectors")
        self._nd=dat.get_hyper().axes[0].n


    def forward(self, add, mod, dat):
        """
        Forward operation
        """
        self.checkDomainRange(mod, dat)
        if not add:
            dat.zero()
        b=0
        for id in range(self._nd):
            b+=mod[id]
            dat[id]+=b


    def adjoint(self, add, mod, dat):
        """
        Adjoint operation.
        """
        self.checkDomainRange(mod, dat)
        if not add:
            mod.zero()

        b=0
        for id in range(self._nd-1,-1,-1):
            b+=dat[id]
            mod[id]+=b


from sep_plot import Dots
import numpy as np
from sep_python import get_sep_vector
import holoviews as hv

inp=get_sep_vector(np.zeros((20),np.float32))

cop=Causal(inp,inp)
p1=inp.clone()
p2=inp.clone()
inp[10]=1
cop.forward(False,inp,p1)
cop.adjoint(False,p2,inp)
hv.Layout(Dots(inp)+Dots(p1)+Dots(p2)).cols(1)

## Leaky integration

The second example is also uses recursion but instead of applying a 1 we are using a leaky integration term. Take note of
applying this approach on a spike.

In [35]:
class Leaky(Operator):

    def __init__(self, mod, dat,p):
        """
        Initialize a transient convolition

            mod, dat - sepVector
        """
        super().__init__(mod, dat)
        if not isinstance(mod, FloatVector) or not isinstance(dat,FloatVector):
            raise Exception("Expecting model, data, flt to be sepVectors")
        self._nd=dat.get_hyper().axes[0].n
        self._p=p


    def forward(self, add, mod, dat):
        """
        Forward operation
        """
        self.checkDomainRange(mod, dat)
        if not add:
            dat.zero()
        b=0
        for id in range(self._nd):
            b=b*self._p+mod[id]
            dat[id]+=b


    def adjoint(self, add, mod, dat):
        """
        Adjoint operation.
        """
        self.checkDomainRange(mod, dat)
        if not add:
            mod.zero()

        b=0
        for id in range(self._nd-1,-1,-1):
            b=b*self._p+dat[id]
            mod[id]+=b

In [36]:
from sep_plot import Dots
import numpy as np
from sep_python import get_sep_vector
import holoviews as hv

inp=get_sep_vector(np.zeros((20),np.float32))

cop=Leaky(inp,inp,.9)
p1=inp.clone()
p2=inp.clone()
inp[10]=1
cop.forward(False,inp,p1)
cop.adjoint(False,p2,inp)
hv.Layout(Dots(inp)+Dots(p1)+Dots(p2)).cols(1)

/Users/clapp/sep-environ/lib/python3.9/site-packages/holoviews/plotting/bokeh/plot.py:987: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  layout_plot = gridplot(
/Users/clapp/sep-environ/lib/python3.9/site-packages/holoviews/plotting/bokeh/plot.py:987: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  layout_plot = gridplot(


:Layout
   .Overlay.I   :Overlay
      .Spikes.I  :Spikes   [x]   (y)
      .Scatter.I :Scatter   [x]   (y)
   .Overlay.II  :Overlay
      .Spikes.I  :Spikes   [x]   (y)
      .Scatter.I :Scatter   [x]   (y)
   .Overlay.III :Overlay
      .Spikes.I  :Spikes   [x]   (y)
      .Scatter.I :Scatter   [x]   (y)

## Boxcar
Our third apporach is to apply a boxcar (something that averages nearby points) to our data. 

In [104]:
class BoxcarF(Operator):

    def __init__(self, mod, dat,halflen):
        """
        Initialize a boxcar convolution (smoothing)

            mod, dat - sepVector
            halflen - Half length of smoothing box
        """
        super().__init__(mod, dat)
        if not isinstance(mod, FloatVector) or not isinstance(dat,FloatVector):
            raise Exception("Expecting model, data, flt to be sepVectors")
        self._halflen=halflen
        self._nd=dat.get_hyper().axes[0].n


    def forward(self, add, mod, dat):
        """
        Forward operation
        """
        self.checkDomainRange(mod, dat)
        if not add:
            dat.zero()
        sc=1./(1+2.*self._halflen)
        for i in range(self._nd):
            tmp=0
            for ib in range(i-self._halflen,i+self._halflen+1):
                tmp+=mod[max(0,min(self._nd-1,ib))]
            dat[i]+=tmp/sc


    def adjoint(self, add, mod, dat):
        """
        Adjoint operation.
        """
        self.checkDomainRange(mod, dat)
        if not add:
            mod.zero()
        
        sc=1./(1+2.*self._halflen)
        for i in range(self._nd):
            tmp=0
            for ib in range(i-self._halflen,i+self._halflen+1):
                mod[max(0,min(self._nd-1,ib))]+=dat[i]/sc
    


In [105]:
from sep_plot import Dots
import numpy as np
from sep_python import get_sep_vector
import holoviews as hv

inp=get_sep_vector(np.zeros((20),np.float32))

cop=BoxcarF(inp,inp,3)
p1=inp.clone()
p2=inp.clone()
inp[10]=1
cop.forward(False,inp,p1)
cop.adjoint(False,p2,inp)
hv.Layout(Dots(inp)+Dots(p1)+Dots(p2)).cols(1)

1 0 7 0.0
2 1 8 0.0
3 2 9 0.0
4 3 10 0.0
5 4 11 0.0
6 5 12 0.0
7 6 13 0.14285715
8 7 14 0.14285715
9 8 15 0.14285715
10 9 16 0.14285715
11 10 17 0.14285715
12 11 18 0.14285715
13 12 19 0.14285715
14 13 20 0.0
15 14 21 0.0
16 15 22 0.0
17 16 23 0.0
18 17 24 0.0
19 18 25 0.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
18 18 25 0.0
17 17 24 0.0
16 16 23 0.0
15 15 22 0.0
14 14 21 0.0
13 13 20 0.14285715
12 12 19 0.14285715
11 11 18 0.14285715
10 10 17 0.14285715
9 9 16 0.14285715
8 8 15 0.14285715
7 7 14 0.14285715
6 6 13 0.0
5 5 12 0.0
4 4 11 0.0
3 3 10 0.0
2 2 9 0.0
1 1 8 0.0
0 0 7 0.0


/Users/clapp/sep-environ/lib/python3.9/site-packages/holoviews/plotting/bokeh/plot.py:987: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  layout_plot = gridplot(
/Users/clapp/sep-environ/lib/python3.9/site-packages/holoviews/plotting/bokeh/plot.py:987: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  layout_plot = gridplot(


:Layout
   .Overlay.I   :Overlay
      .Spikes.I  :Spikes   [x]   (y)
      .Scatter.I :Scatter   [x]   (y)
   .Overlay.II  :Overlay
      .Spikes.I  :Spikes   [x]   (y)
      .Scatter.I :Scatter   [x]   (y)
   .Overlay.III :Overlay
      .Spikes.I  :Spikes   [x]   (y)
      .Scatter.I :Scatter   [x]   (y)

In [94]:
from sep_plot import Dots
import numpy as np
from sep_python import get_sep_vector
import holoviews as hv

inp=get_sep_vector(np.zeros((20),np.float32))

cop=BoxcarF(inp,inp,3)
p1=inp.clone()
p2=inp.clone()
inp[10]=1
cop.forward(False,inp,p1)
cop.adjoint(False,p2,inp)
hv.Layout(Dots(inp)+Dots(p1)+Dots(p2)).cols(1)

18 18 25
17 17 24
16 16 23
15 15 22
14 14 21
13 13 20
12 12 19
11 11 18
10 10 17
9 9 16
8 8 15
7 7 14
6 6 13
5 5 12
4 4 11
3 3 10
2 2 9
1 1 8
0 0 7


/Users/clapp/sep-environ/lib/python3.9/site-packages/holoviews/plotting/bokeh/plot.py:987: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  layout_plot = gridplot(
/Users/clapp/sep-environ/lib/python3.9/site-packages/holoviews/plotting/bokeh/plot.py:987: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  layout_plot = gridplot(


:Layout
   .Overlay.I   :Overlay
      .Spikes.I  :Spikes   [x]   (y)
      .Scatter.I :Scatter   [x]   (y)
   .Overlay.II  :Overlay
      .Spikes.I  :Spikes   [x]   (y)
      .Scatter.I :Scatter   [x]   (y)
   .Overlay.III :Overlay
      .Spikes.I  :Spikes   [x]   (y)
      .Scatter.I :Scatter   [x]   (y)